# 🚀 IntelliMatch Pro - AI Career Intelligence Platform

## The Future of Intelligent Talent Matching

**IntelliMatch Pro** is a cutting-edge AI-powered career intelligence platform that revolutionizes how job seekers find opportunities and how employers discover talent. Using advanced machine learning and natural language processing, it creates perfect matches between candidates and positions.

### 🧠 Powered by Advanced AI Technology:
- **Tavily AI** for comprehensive web search and LinkedIn data extraction
- **OpenAI GPT-4o-mini** for intelligent natural language processing and analysis
- **LangChain** for sophisticated AI workflow orchestration
- **LangSmith** for performance monitoring and optimization
- **LangGraph** for complex workflow management
- **Plotly** for stunning interactive visualizations

### ✨ Core Features:
- 🎯 **Career Navigator**: AI-driven job discovery based on skills, experience, and career goals
- ? **Talent Finder**: Advanced candidate sourcing and ranking for recruiters
- 📊 **Intelligence Dashboard**: Real-time analytics on match quality and market trends
- 🧬 **Smart Compatibility Engine**: AI-powered scoring with detailed compatibility insights
- 🚀 **Future-Ready**: Continuously learns and improves matching accuracy

*Transform your career journey or talent acquisition process with IntelliMatch Pro - where artificial intelligence meets human potential.*

## 1. Environment Setup and API Configuration

Setting up environment variables and API keys for all required services.

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configure API Keys from environment variables for security
# Set these in your environment or create a .env file:
# OPENAI_API_KEY=your_openai_api_key_here
# TAVILY_API_KEY=your_tavily_api_key_here

openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

if not openai_key:
    raise ValueError("❌ OPENAI_API_KEY environment variable not found! Please set it before running.")
if not tavily_key:
    raise ValueError("❌ TAVILY_API_KEY environment variable not found! Please set it before running.")

os.environ["OPENAI_API_KEY"] = openai_key
os.environ["TAVILY_API_KEY"] = tavily_key

# Optional: LangSmith configuration (uncomment and add your keys if available)
# os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
# os.environ["LANGCHAIN_PROJECT"] = "intellimatch-pro"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

print("✅ Environment configured successfully!")
print(f"🔑 OpenAI API Key: {'*' * 20}... (secured)")
print(f"🔑 Tavily API Key: {'*' * 20}... (secured)")
print("🔒 API keys are now securely loaded from environment variables!")

## 2. Import Required Libraries and Dependencies

Importing all necessary libraries for the AI job matching agent.

In [ ]:
# Core libraries
import asyncio
import json
import re
from datetime import datetime
from typing import Dict, List, Optional, Union, Any
import traceback

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Jupyter notebook specific
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import HumanMessage, SystemMessage

# LangGraph imports
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# LangSmith imports (optional)
try:
    from langsmith import Client, traceable
    LANGSMITH_AVAILABLE = True
    print("✅ LangSmith available")
except ImportError:
    LANGSMITH_AVAILABLE = False
    print("⚠️ LangSmith not available - continuing without tracing")

print("✅ All libraries imported successfully!")
print("🚀 Ready to build the AI Job Matching Agent!")

## 3. Define Data Models and Schemas

Creating Pydantic models for structured data handling throughout the job matching system.

In [ ]:
class JobSeeker(BaseModel):
    """Model for job seeker profile"""
    name: str
    skills: List[str]
    experience_years: int
    education: str
    location: str
    desired_salary_range: Optional[str] = None
    job_preferences: List[str] = []
    industry_preferences: List[str] = []
    additional_requirements: Optional[str] = None

class JobPosting(BaseModel):
    """Model for job posting information"""
    title: str
    company: str
    location: str
    salary_range: Optional[str] = None
    required_skills: List[str]
    experience_required: str
    job_type: str = "Full-time"
    description: str
    url: Optional[str] = None
    match_score: Optional[float] = None

class Candidate(BaseModel):
    """Model for candidate profile"""
    name: str
    current_position: str
    company: str
    skills: List[str]
    experience: str
    education: str
    location: str
    linkedin_url: str
    match_score: Optional[float] = None
    contact_info: Optional[str] = None

class MatchResult(BaseModel):
    """Model for match analysis results"""
    total_score: float
    skills_score: float
    experience_score: float
    location_score: float
    education_score: float
    strengths: List[str]
    gaps: List[str]
    recommendations: List[str]

class WorkflowState(TypedDict):
    """State model for LangGraph workflow"""
    request_type: str
    user_input: dict
    search_results: list
    processed_data: dict
    matches: list
    recommendations: list
    final_output: dict
    error_message: Optional[str]

print("✅ Data models defined successfully!")
print("📊 Models include: JobSeeker, JobPosting, Candidate, MatchResult, WorkflowState")

## 4. Create Agent Prompt Templates

Developing specialized prompts for job seeker assistance and recruiter assistance.

In [ ]:
# Job Seeker Assistant Prompt Template
JOB_SEEKER_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """
You are an expert career advisor and job matching specialist. Your role is to help job seekers find the most suitable positions based on their profile, skills, and preferences.

CORE RESPONSIBILITIES:
1. Analyze job seeker's profile comprehensively
2. Search for relevant job opportunities on LinkedIn and other platforms
3. Match skills and experience with job requirements
4. Provide personalized job recommendations with detailed explanations
5. Suggest skill improvements and career development paths
6. Offer interview preparation and application advice

ANALYSIS FRAMEWORK:
- Skills Assessment: Technical, soft skills, and domain expertise
- Experience Evaluation: Years, projects, achievements, career progression
- Cultural Fit: Company values, work environment preferences
- Growth Potential: Career advancement opportunities, learning prospects
- Compensation Alignment: Salary expectations vs market rates
- Location Preferences: Remote, hybrid, on-site requirements

SEARCH STRATEGY:
- Use advanced LinkedIn search techniques with Boolean operators
- Search multiple job boards and company career pages
- Filter by experience level, location, company size, industry
- Identify both direct applications and networking opportunities
- Track application success rates and optimize search terms

OUTPUT FORMAT:
Provide structured recommendations including:
- Job match score (0-100) with detailed reasoning
- Required vs possessed skills gap analysis
- Specific application advice for each opportunity
- Timeline and priority ranking
- Alternative career paths if direct matches are limited
"""),
    ("human", """
Job Seeker Profile:
Name: {name}
Current Skills: {skills}
Experience: {experience_years} years
Education: {education}
Location: {location}
Desired Salary: {desired_salary}
Job Preferences: {job_preferences}
Industry Preferences: {industry_preferences}
Additional Requirements: {additional_requirements}

Based on the LinkedIn search results: {search_results}

Please analyze and provide personalized job recommendations for this candidate.
""")
])

# Recruiter Assistant Prompt Template
RECRUITER_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """
You are an expert technical recruiter and talent acquisition specialist. Your mission is to find the best candidates for employers' specific job requirements using advanced sourcing techniques.

CORE RESPONSIBILITIES:
1. Analyze job requirements and create detailed candidate personas
2. Execute comprehensive LinkedIn talent searches
3. Evaluate candidate profiles against specific criteria
4. Rank candidates based on fit, experience, and potential
5. Provide sourcing strategies and outreach recommendations
6. Suggest interview questions and assessment criteria

CANDIDATE SOURCING FRAMEWORK:
- Technical Skills Matching: Exact match, transferable skills, learning potential
- Experience Relevance: Industry experience, company size, role complexity
- Cultural Assessment: Values alignment, team fit, communication style
- Availability Indicators: Job change signals, career progression timing
- Diversity and Inclusion: Balanced candidate pool considerations
- Passive vs Active: Identify both job seekers and passive candidates

SEARCH METHODOLOGY:
- Advanced Boolean search on LinkedIn with multiple keyword combinations
- X-ray searching through Google for additional profiles
- University alumni networks and professional associations
- GitHub, Stack Overflow, and technical community searches
- Competitive intelligence and talent mapping
- Social media professional presence analysis

EVALUATION CRITERIA:
- Technical competency assessment (1-10 scale)
- Cultural fit probability (1-10 scale)
- Motivation and career goals alignment
- Salary expectations vs budget alignment
- Availability and notice period considerations
- Reference and background check likelihood

OUTPUT STRUCTURE:
- Ranked candidate list with detailed profiles
- Match score breakdown and reasoning
- Outreach strategy for each candidate tier
- Alternative sourcing channels if needed
- Market intelligence and salary benchmarking
"""),
    ("human", """
Job Requirements:
Position: {job_title}
Company: {company_name}
Location: {job_location}
Experience Required: {experience_required}
Must-Have Skills: {required_skills}
Nice-to-Have Skills: {preferred_skills}
Education Requirements: {education_requirements}
Salary Range: {salary_range}
Company Culture: {company_culture}
Special Requirements: {special_requirements}

Based on the LinkedIn search results: {search_results}

Please analyze and rank the best candidates for this position.
""")
])

print("✅ Prompt templates created successfully!")
print("🎯 Templates include: Job Seeker Assistant and Recruiter Assistant")

In [ ]:
async def search_linkedin_profiles(state: WorkflowState) -> Dict:
    """Search LinkedIn for relevant profiles/jobs"""
    try:
        processed_data = state["processed_data"]
        request_type = state["request_type"]
        
        print(f"🔍 Searching LinkedIn for {request_type} data...")
        
        if request_type == "job_seeker":
            # Create JobSeeker object for the search
            job_seeker = JobSeeker(
                name=processed_data["name"],
                skills=processed_data["skills"],
                experience_years=processed_data["experience_years"],
                education=processed_data.get("education", ""),
                location=processed_data["location"],
                desired_salary_range=processed_data.get("desired_salary", ""),
                job_preferences=processed_data.get("job_preferences", []),
                industry_preferences=processed_data.get("industry_preferences", []),
                additional_requirements=processed_data.get("additional_requirements", "")
            )
            
            # Generate search queries for job search
            queries = search_engine.generate_job_search_queries(job_seeker)
            
            # Search for job postings
            results = await search_engine.search_linkedin_jobs(queries)
            
        else:  # recruiter
            # Use processed_data directly for candidate search
            queries = search_engine.generate_candidate_search_queries(processed_data)
            
            # Search for candidate profiles
            results = await search_engine.search_linkedin_candidates(queries)
        
        print(f"✅ Found {len(results)} results from LinkedIn")
        return {"search_results": results}
    
    except Exception as e:
        print(f"❌ Error searching LinkedIn: {str(e)}")
        import traceback
        print("Full traceback:")
        traceback.print_exc()
        return {"error_message": f"Error searching LinkedIn: {str(e)}"}

In [ ]:
async def enhance_search_data(state: WorkflowState) -> Dict:
    """Enhance raw search results with AI analysis and structured data extraction"""
    try:
        search_results = state.get("search_results", [])
        request_type = state["request_type"]
        
        if not search_results:
            print("⚠️ No search results to enhance")
            return {"enhanced_data": []}
        
        print(f"🧠 Enhancing search results with AI analysis...")
        
        enhanced_results = []
        
        # Process up to 20 results for detailed analysis
        for result in search_results[:20]:
            try:
                if request_type == "job_seeker":
                    # Add basic job structure to the result
                    enhanced_job = {
                        "title": result.get("title", "Unknown Position"),
                        "company": result.get("company", "Unknown Company"),
                        "location": result.get("location", "Unknown Location"),
                        "url": result.get("url", ""),
                        "content": result.get("content", ""),
                        "required_skills": [],  # Will be extracted later
                        "experience_required": "",
                        "salary_range": "",
                        "source": result.get("source", "LinkedIn")
                    }
                    enhanced_results.append(enhanced_job)
                    
                else:  # recruiter
                    # Add basic candidate structure to the result
                    enhanced_candidate = {
                        "name": search_engine.extract_name_from_title(result.get("title", "")),
                        "current_position": "Unknown Position",
                        "company": "Unknown Company",
                        "skills": [],  # Will be extracted later
                        "experience": "Unknown",
                        "education": "Unknown",
                        "location": "Unknown Location",
                        "linkedin_url": result.get("url", ""),
                        "content": result.get("content", "")
                    }
                    enhanced_results.append(enhanced_candidate)
                    
            except Exception as e:
                print(f"Warning: Failed to enhance result: {str(e)}")
                # Add the original result if enhancement fails
                enhanced_results.append(result)
        
        print(f"✅ Enhanced {len(enhanced_results)} results")
        return {"enhanced_data": enhanced_results}
    
    except Exception as e:
        print(f"❌ Error enhancing search data: {str(e)}")
        return {"error_message": f"Error enhancing search data: {str(e)}"}

## 5. Implement LinkedIn Search Functions

Building functions to generate optimized search queries and execute LinkedIn searches using Tavily AI.

In [ ]:
class LinkedInSearchEngine:
    """Advanced LinkedIn search engine using Tavily AI"""
    
    def __init__(self):
        self.tavily = TavilySearchResults(max_results=10)
        self.search_cache = {}
    
    def generate_job_search_queries(self, job_seeker: JobSeeker) -> List[str]:
        """Generate optimized search queries for job opportunities"""
        queries = []
        
        # Primary skill-based searches
        for skill in job_seeker.skills[:5]:  # Top 5 skills
            query = f'"{skill}" jobs {job_seeker.location}'
            queries.append(query)
        
        # Industry and role combination searches
        for industry in job_seeker.industry_preferences[:3]:
            for preference in job_seeker.job_preferences[:2]:
                query = f'{industry} {preference} {job_seeker.location}'
                queries.append(query)
        
        # Experience level searches
        exp_level = self.categorize_experience(job_seeker.experience_years)
        for skill in job_seeker.skills[:3]:
            query = f'{exp_level} {skill} opportunities'
            queries.append(query)
        
        return queries[:8]  # Limit to 8 queries for efficiency
    
    def generate_candidate_search_queries(self, job_requirements: Dict) -> List[str]:
        """Generate optimized search queries for candidate sourcing"""
        queries = []
        
        # Skill-based searches
        required_skills = job_requirements.get("required_skills", [])
        for skill in required_skills[:5]:
            queries.append(f'"{skill}" {job_requirements.get("job_location", "")}')
            queries.append(f'"{skill}" "{job_requirements.get("job_title", "")}"')
        
        # Boolean combinations for multiple skills
        if len(required_skills) >= 2:
            skill_combo = " AND ".join(f'"{skill}"' for skill in required_skills[:3])
            queries.append(skill_combo)
        
        # Experience and title combinations
        title = job_requirements.get("job_title", "")
        if title:
            queries.append(f'"{title}" {job_requirements.get("experience_required", "")}')
        
        return queries[:8]  # Limit to 8 queries for efficiency
    
    def categorize_experience(self, years: int) -> str:
        """Categorize experience level"""
        if years <= 2:
            return "entry level junior"
        elif years <= 5:
            return "mid level"
        elif years <= 10:
            return "senior"
        else:
            return "executive principal"
    
    async def search_linkedin_jobs(self, queries: List[str]) -> List[Dict]:
        """Search for job postings on LinkedIn"""
        all_results = []
        
        for query in queries:
            try:
                # Search specifically on LinkedIn for jobs
                linkedin_query = f"site:linkedin.com/jobs {query}"
                results = self.tavily.run(linkedin_query)
                
                # Process and clean results
                for result in results:
                    processed_result = {
                        "title": result.get("title", "Unknown Position"),
                        "url": result.get("url", ""),
                        "content": result.get("content", ""),
                        "source": "LinkedIn Jobs",
                        "query_used": query
                    }
                    all_results.append(processed_result)
                
                # Add small delay to respect rate limits
                await asyncio.sleep(0.5)
                
            except Exception as e:
                print(f"Error searching for query '{query}': {str(e)}")
                continue
        
        return all_results
    
    async def search_linkedin_candidates(self, queries: List[str]) -> List[Dict]:
        """Search for candidate profiles on LinkedIn"""
        all_results = []
        
        for query in queries:
            try:
                # Search specifically on LinkedIn for profiles
                linkedin_query = f"site:linkedin.com/in {query}"
                results = self.tavily.run(linkedin_query)
                
                # Process and clean results
                for result in results:
                    processed_result = {
                        "name": self.extract_name_from_title(result.get("title", "")),
                        "url": result.get("url", ""),
                        "content": result.get("content", ""),
                        "profile_summary": result.get("title", ""),
                        "source": "LinkedIn Profiles",
                        "query_used": query
                    }
                    all_results.append(processed_result)
                
                # Add small delay to respect rate limits
                await asyncio.sleep(0.5)
                
            except Exception as e:
                print(f"Error searching for query '{query}': {str(e)}")
                continue
        
        return all_results
    
    def extract_name_from_title(self, title: str) -> str:
        """Extract name from LinkedIn profile title"""
        # LinkedIn titles usually start with the person's name
        if " - " in title:
            return title.split(" - ")[0].strip()
        elif " | " in title:
            return title.split(" | ")[0].strip()
        else:
            # Take first few words as name
            words = title.split()[:2]
            return " ".join(words)

# Initialize the search engine
search_engine = LinkedInSearchEngine()

print("✅ LinkedIn Search Engine initialized successfully!")
print("🔍 Ready to search for jobs and candidates on LinkedIn via Tavily AI")

## 6. Build Matching Algorithm Engine

Implementing advanced algorithms to calculate compatibility scores between candidates and opportunities.

In [ ]:
class MatchingEngine:
    """Advanced matching algorithm for job-candidate pairing"""
    
    def __init__(self):
        self.skill_weights = {
            "required": 1.0,
            "preferred": 0.7,
            "transferable": 0.5
        }
        self.experience_weights = {
            "exact": 1.0,
            "related": 0.8,
            "transferable": 0.6
        }
        
    def calculate_skills_match(self, candidate_skills: List[str], 
                             required_skills: List[str], 
                             preferred_skills: List[str] = None) -> float:
        """Calculate skills matching score"""
        if not candidate_skills or not required_skills:
            return 0.0
            
        # Normalize skills for comparison
        candidate_skills_norm = [skill.lower().strip() for skill in candidate_skills]
        required_skills_norm = [skill.lower().strip() for skill in required_skills]
        preferred_skills_norm = [skill.lower().strip() for skill in (preferred_skills or [])]
        
        # Calculate required skills match
        required_matches = 0
        for req_skill in required_skills_norm:
            for cand_skill in candidate_skills_norm:
                if (req_skill in cand_skill or cand_skill in req_skill or 
                    self.are_similar_skills(req_skill, cand_skill)):
                    required_matches += 1
                    break
        
        required_score = required_matches / len(required_skills_norm) if required_skills_norm else 0
        
        # Calculate preferred skills match
        preferred_score = 0
        if preferred_skills_norm:
            preferred_matches = 0
            for pref_skill in preferred_skills_norm:
                for cand_skill in candidate_skills_norm:
                    if (pref_skill in cand_skill or cand_skill in pref_skill or 
                        self.are_similar_skills(pref_skill, cand_skill)):
                        preferred_matches += 1
                        break
            preferred_score = preferred_matches / len(preferred_skills_norm)
        
        # Combined score with weights
        return (required_score * 0.8) + (preferred_score * 0.2)
    
    def are_similar_skills(self, skill1: str, skill2: str) -> bool:
        """Check if two skills are similar"""
        # Simple similarity check - can be enhanced with NLP
        skill_synonyms = {
            "js": "javascript",
            "ts": "typescript", 
            "react": "reactjs",
            "node": "nodejs",
            "py": "python",
            "ml": "machine learning",
            "ai": "artificial intelligence",
            "aws": "amazon web services"
        }
        
        # Normalize and check synonyms
        skill1_norm = skill_synonyms.get(skill1, skill1)
        skill2_norm = skill_synonyms.get(skill2, skill2)
        
        return skill1_norm == skill2_norm or skill1 in skill2 or skill2 in skill1
    
    def calculate_experience_match(self, candidate_experience: Union[str, int], 
                                 required_experience: str) -> float:
        """Calculate experience matching score"""
        try:
            # Extract years from strings
            if isinstance(candidate_experience, str):
                cand_years = self.extract_years_from_text(candidate_experience)
            else:
                cand_years = candidate_experience
            
            req_years = self.extract_years_from_text(required_experience)
            
            if cand_years >= req_years:
                return 1.0
            elif cand_years >= req_years * 0.8:
                return 0.8
            elif cand_years >= req_years * 0.6:
                return 0.6
            else:
                return 0.3
                
        except:
            return 0.5  # Default score if parsing fails
    
    def extract_years_from_text(self, text: str) -> int:
        """Extract years of experience from text"""
        # Look for patterns like "5 years", "3+ years", "5-7 years"
        import re
        
        patterns = [
            r'(\d+)\+?\s*years?',
            r'(\d+)-\d+\s*years?',
            r'(\d+)\s*yrs?',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                return int(match.group(1))
        
        return 0
    
    def calculate_location_match(self, candidate_location: str, 
                               job_location: str) -> float:
        """Calculate location matching score"""
        if not candidate_location or not job_location:
            return 0.5
        
        cand_loc = candidate_location.lower().strip()
        job_loc = job_location.lower().strip()
        
        # Check for remote work
        if "remote" in job_loc:
            return 1.0
        
        # Check for same city/state
        if cand_loc in job_loc or job_loc in cand_loc:
            return 1.0
        
        # Check for same state (basic implementation)
        cand_state = cand_loc.split(",")[-1].strip() if "," in cand_loc else ""
        job_state = job_loc.split(",")[-1].strip() if "," in job_loc else ""
        
        if cand_state and job_state and cand_state == job_state:
            return 0.7
        
        return 0.3
    
    def calculate_education_match(self, candidate_education: str, 
                                required_education: str) -> float:
        """Calculate education matching score"""
        if not required_education:
            return 1.0
        
        if not candidate_education:
            return 0.5
        
        cand_edu = candidate_education.lower()
        req_edu = required_education.lower()
        
        # Check for degree levels
        degree_hierarchy = {
            "phd": 4, "doctorate": 4,
            "masters": 3, "ms": 3, "mba": 3, "ma": 3,
            "bachelors": 2, "bs": 2, "ba": 2, "bsc": 2,
            "associates": 1, "aa": 1,
            "certificate": 1, "diploma": 1
        }
        
        cand_level = 0
        req_level = 0
        
        for degree, level in degree_hierarchy.items():
            if degree in cand_edu:
                cand_level = max(cand_level, level)
            if degree in req_edu:
                req_level = max(req_level, level)
        
        if cand_level >= req_level:
            return 1.0
        elif cand_level == req_level - 1:
            return 0.8
        else:
            return 0.6
    
    def calculate_comprehensive_match(self, candidate_data: Dict, 
                                    requirements: Dict) -> MatchResult:
        """Calculate comprehensive match score with detailed breakdown"""
        
        # Extract data safely
        candidate_skills = candidate_data.get("skills", [])
        required_skills = requirements.get("required_skills", [])
        preferred_skills = requirements.get("preferred_skills", [])
        
        # Calculate individual scores
        skills_score = self.calculate_skills_match(
            candidate_skills, required_skills, preferred_skills
        )
        
        experience_score = self.calculate_experience_match(
            candidate_data.get("experience", ""),
            requirements.get("experience_required", "")
        )
        
        location_score = self.calculate_location_match(
            candidate_data.get("location", ""),
            requirements.get("job_location", "")
        )
        
        education_score = self.calculate_education_match(
            candidate_data.get("education", ""),
            requirements.get("education_requirements", "")
        )
        
        # Calculate weighted total
        total_score = (
            skills_score * 0.40 +      # Skills are most important
            experience_score * 0.30 +   # Experience is crucial
            location_score * 0.15 +     # Location matters
            education_score * 0.15      # Education is valuable
        )
        
        # Generate insights
        strengths = self.identify_strengths(candidate_data, requirements, {
            "skills": skills_score,
            "experience": experience_score,
            "location": location_score,
            "education": education_score
        })
        
        gaps = self.identify_gaps(candidate_data, requirements, {
            "skills": skills_score,
            "experience": experience_score,
            "location": location_score,
            "education": education_score
        })
        
        recommendations = self.generate_recommendations(total_score, strengths, gaps)
        
        return MatchResult(
            total_score=round(total_score * 100, 1),
            skills_score=round(skills_score * 100, 1),
            experience_score=round(experience_score * 100, 1),
            location_score=round(location_score * 100, 1),
            education_score=round(education_score * 100, 1),
            strengths=strengths,
            gaps=gaps,
            recommendations=recommendations
        )
    
    def identify_strengths(self, candidate_data: Dict, requirements: Dict, scores: Dict) -> List[str]:
        """Identify candidate's strengths"""
        strengths = []
        
        if scores["skills"] > 0.8:
            strengths.append("Strong technical skill alignment")
        if scores["experience"] > 0.8:
            strengths.append("Excellent experience match")
        if scores["location"] > 0.8:
            strengths.append("Ideal location fit")
        if scores["education"] > 0.8:
            strengths.append("Strong educational background")
        
        return strengths
    
    def identify_gaps(self, candidate_data: Dict, requirements: Dict, scores: Dict) -> List[str]:
        """Identify areas for improvement"""
        gaps = []
        
        if scores["skills"] < 0.6:
            gaps.append("Some required skills missing")
        if scores["experience"] < 0.6:
            gaps.append("Experience level below requirements")
        if scores["location"] < 0.6:
            gaps.append("Location may require relocation")
        if scores["education"] < 0.6:
            gaps.append("Educational requirements not fully met")
        
        return gaps
    
    def generate_recommendations(self, total_score: float, strengths: List[str], gaps: List[str]) -> List[str]:
        """Generate actionable recommendations"""
        recommendations = []
        
        if total_score > 0.8:
            recommendations.append("Excellent candidate - proceed with interview")
        elif total_score > 0.6:
            recommendations.append("Good candidate - consider for next round")
        else:
            recommendations.append("May need additional evaluation")
        
        if len(gaps) > 0:
            recommendations.append("Address skill gaps through training or mentoring")
        
        return recommendations

# Initialize matching engine
matching_engine = MatchingEngine()

print("✅ Matching Engine initialized successfully!")
print("🎯 Ready to calculate compatibility scores and match insights")

## 7. Develop LangGraph Workflow

Creating a comprehensive workflow that orchestrates the entire job matching process.

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# Workflow Node Functions
async def process_user_input(state: WorkflowState) -> Dict:
    """Process and validate user input"""
    try:
        request_type = state["request_type"]
        user_input = state["user_input"]
        
        print(f"📝 Processing {request_type} request...")
        
        # Validate and structure input based on request type
        if request_type == "job_seeker":
            processed_input = {
                "name": user_input.get("name", "Anonymous"),
                "skills": user_input.get("skills", []),
                "experience_years": user_input.get("experience_years", 0),
                "education": user_input.get("education", ""),
                "location": user_input.get("location", ""),
                "desired_salary": user_input.get("desired_salary", ""),
                "job_preferences": user_input.get("job_preferences", []),
                "industry_preferences": user_input.get("industry_preferences", []),
                "additional_requirements": user_input.get("additional_requirements", "")
            }
        else:  # recruiter
            processed_input = {
                "job_title": user_input.get("job_title", ""),
                "company_name": user_input.get("company_name", ""),
                "job_location": user_input.get("job_location", ""),
                "experience_required": user_input.get("experience_required", ""),
                "required_skills": user_input.get("required_skills", []),
                "preferred_skills": user_input.get("preferred_skills", []),
                "education_requirements": user_input.get("education_requirements", ""),
                "salary_range": user_input.get("salary_range", ""),
                "company_culture": user_input.get("company_culture", ""),
                "special_requirements": user_input.get("special_requirements", "")
            }
        
        return {"processed_data": processed_input}
    
    except Exception as e:
        return {"error_message": f"Error processing input: {str(e)}"}

async def search_linkedin_profiles(state: WorkflowState) -> Dict:
    """Execute LinkedIn searches using Tavily AI"""
    try:
        request_type = state["request_type"]
        processed_data = state["processed_data"]
        
        print(f"🔍 Searching LinkedIn for {request_type} data...")
        
        if request_type == "job_seeker":
            # Create JobSeeker object for query generation
            job_seeker = JobSeeker(
                name=processed_data["name"],
                skills=processed_data["skills"],
                experience_years=processed_data["experience_years"],
                education=processed_data["education"],
                location=processed_data["location"],
                desired_salary_range=processed_data["desired_salary"],
                job_preferences=processed_data["job_preferences"],
                industry_preferences=processed_data["industry_preferences"],
                additional_requirements=processed_data["additional_requirements"]
            )
            
            # Generate search queries and search for jobs
            queries = search_engine.generate_job_search_queries(job_seeker)
            search_results = await search_engine.search_linkedin_jobs(queries)
            
        else:  # recruiter
            # Generate search queries and search for candidates
            queries = search_engine.generate_candidate_search_queries(processed_data)
            search_results = await search_engine.search_linkedin_candidates(queries)
        
        print(f"✅ Found {len(search_results)} results from LinkedIn")
        return {"search_results": search_results}
    
    except Exception as e:
        return {"error_message": f"Error searching LinkedIn: {str(e)}"}

async def enhance_search_data(state: WorkflowState) -> Dict:
    """Enhance search results with AI analysis"""
    try:
        search_results = state["search_results"]
        request_type = state["request_type"]
        
        print("🧠 Enhancing search results with AI analysis...")
        
        enhanced_results = []
        
        for result in search_results[:20]:  # Limit processing for efficiency
            try:
                # Extract relevant information from content using AI
                content = result.get("content", "")
                
                if request_type == "job_seeker":
                    # Extract job information
                    enhanced_data = await extract_job_information(content, result)
                else:
                    # Extract candidate information
                    enhanced_data = await extract_candidate_information(content, result)
                
                enhanced_results.append(enhanced_data)
                
            except Exception as e:
                print(f"Error enhancing result: {str(e)}")
                # Keep original result if enhancement fails
                enhanced_results.append(result)
        
        print(f"✅ Enhanced {len(enhanced_results)} results")
        return {"enhanced_results": enhanced_results}
    
    except Exception as e:
        return {"error_message": f"Error enhancing search data: {str(e)}"}

async def extract_job_information(content: str, original_result: Dict) -> Dict:
    """Extract structured job information using AI"""
    extraction_prompt = f"""
    Extract job information from this LinkedIn content and return a JSON structure:
    
    Content: {content[:1000]}...
    
    Extract:
    - Job title
    - Company name
    - Location
    - Required skills (as list)
    - Experience requirements
    - Salary information (if available)
    - Job type (full-time, part-time, contract, etc.)
    - Key responsibilities
    
    Return only valid JSON format.
    """
    
    try:
        response = await llm.ainvoke([HumanMessage(content=extraction_prompt)])
        
        # Try to parse JSON response
        try:
            extracted_data = json.loads(response.content)
        except:
            # If JSON parsing fails, create basic structure
            extracted_data = {
                "title": original_result.get("title", "Unknown Position"),
                "company": "Unknown Company",
                "location": "Unknown Location",
                "required_skills": [],
                "experience_required": "Not specified",
                "salary_range": None,
                "job_type": "Full-time",
                "description": content[:500]
            }
        
        # Merge with original result
        extracted_data.update({
            "url": original_result.get("url", ""),
            "source": original_result.get("source", "LinkedIn"),
            "raw_content": content
        })
        
        return extracted_data
        
    except Exception as e:
        # Fallback structure
        return {
            "title": original_result.get("title", "Unknown Position"),
            "company": "Unknown Company",
            "location": "Unknown Location",
            "required_skills": [],
            "experience_required": "Not specified",
            "url": original_result.get("url", ""),
            "source": "LinkedIn",
            "raw_content": content
        }

async def extract_candidate_information(content: str, original_result: Dict) -> Dict:
    """Extract structured candidate information using AI"""
    extraction_prompt = f"""
    Extract candidate information from this LinkedIn profile content and return a JSON structure:
    
    Content: {content[:1000]}...
    
    Extract:
    - Full name
    - Current position/title
    - Current company
    - Skills (as list)
    - Years of experience
    - Education
    - Location
    - Previous companies (if mentioned)
    
    Return only valid JSON format.
    """
    
    try:
        response = await llm.ainvoke([HumanMessage(content=extraction_prompt)])
        
        # Try to parse JSON response
        try:
            extracted_data = json.loads(response.content)
        except:
            # If JSON parsing fails, create basic structure
            extracted_data = {
                "name": original_result.get("name", "Unknown Candidate"),
                "current_position": "Unknown Position",
                "company": "Unknown Company",
                "skills": [],
                "experience": "Unknown",
                "education": "Unknown",
                "location": "Unknown Location"
            }
        
        # Merge with original result
        extracted_data.update({
            "linkedin_url": original_result.get("url", ""),
            "profile_summary": original_result.get("profile_summary", ""),
            "raw_content": content
        })
        
        return extracted_data
        
    except Exception as e:
        # Fallback structure
        return {
            "name": original_result.get("name", "Unknown Candidate"),
            "current_position": "Unknown Position",
            "company": "Unknown Company",
            "skills": [],
            "experience": "Unknown",
            "education": "Unknown",
            "location": "Unknown Location",
            "linkedin_url": original_result.get("url", ""),
            "raw_content": content
        }

async def calculate_matches(state: WorkflowState) -> Dict:
    """Calculate match scores between requirements and results"""
    try:
        enhanced_results = state["enhanced_results"]
        processed_data = state["processed_data"]
        request_type = state["request_type"]
        
        print("🎯 Calculating match scores...")
        
        matches = []
        
        for result in enhanced_results:
            try:
                if request_type == "job_seeker":
                    # Calculate job match for job seeker
                    match_result = matching_engine.calculate_comprehensive_match(
                        candidate_data={
                            "skills": processed_data["skills"],
                            "experience": str(processed_data["experience_years"]) + " years",
                            "location": processed_data["location"],
                            "education": processed_data["education"]
                        },
                        requirements={
                            "required_skills": result.get("required_skills", []),
                            "preferred_skills": [],
                            "experience_required": result.get("experience_required", ""),
                            "job_location": result.get("location", ""),
                            "education_requirements": ""
                        }
                    )
                else:
                    # Calculate candidate match for recruiter
                    match_result = matching_engine.calculate_comprehensive_match(
                        candidate_data={
                            "skills": result.get("skills", []),
                            "experience": result.get("experience", ""),
                            "location": result.get("location", ""),
                            "education": result.get("education", "")
                        },
                        requirements=processed_data
                    )
                
                # Add match score to result
                result["match_score"] = match_result.total_score
                result["match_details"] = match_result.dict()
                
                matches.append(result)
                
            except Exception as e:
                print(f"Error calculating match for result: {str(e)}")
                result["match_score"] = 0
                matches.append(result)
        
        # Sort by match score
        matches.sort(key=lambda x: x.get("match_score", 0), reverse=True)
        
        print(f"✅ Calculated matches for {len(matches)} results")
        return {"matches": matches}
    
    except Exception as e:
        return {"error_message": f"Error calculating matches: {str(e)}"}

async def generate_recommendations(state: WorkflowState) -> Dict:
    """Generate AI-powered recommendations"""
    try:
        # Check if matches exist
        if "matches" not in state:
            print("⚠️ No matches found in state, checking for enhanced_data...")
            # Try to use enhanced_data if matches failed
            if "enhanced_data" in state:
                matches = state["enhanced_data"][:10]  # Use first 10 enhanced results
                print(f"✅ Using {len(matches)} enhanced results as fallback")
            else:
                print("❌ No data available for recommendations")
                return {"error_message": "No match data available for recommendations"}
        else:
            matches = state["matches"]
        
        processed_data = state["processed_data"]
        request_type = state["request_type"]
        
        print("💡 Generating AI recommendations...")
        
        # Select top matches for detailed analysis
        top_matches = matches[:10]
        
        if request_type == "job_seeker":
            # Generate job recommendations
            prompt = JOB_SEEKER_PROMPT.format(
                name=processed_data["name"],
                skills=", ".join(processed_data["skills"]),
                experience_years=processed_data["experience_years"],
                education=processed_data.get("education", ""),
                location=processed_data["location"],
                desired_salary=processed_data.get("desired_salary", ""),
                job_preferences=", ".join(processed_data.get("job_preferences", [])),
                industry_preferences=", ".join(processed_data.get("industry_preferences", [])),
                additional_requirements=processed_data.get("additional_requirements", ""),
                search_results=json.dumps(top_matches, indent=2)
            )
        else:
            # Generate candidate recommendations
            prompt = RECRUITER_PROMPT.format(
                job_title=processed_data.get("job_title", ""),
                company_name=processed_data.get("company_name", ""),
                job_location=processed_data.get("job_location", ""),
                experience_required=processed_data.get("experience_required", ""),
                required_skills=", ".join(processed_data.get("required_skills", [])),
                preferred_skills=", ".join(processed_data.get("preferred_skills", [])),
                education_requirements=processed_data.get("education_requirements", ""),
                salary_range=processed_data.get("salary_range", ""),
                company_culture=processed_data.get("company_culture", ""),
                special_requirements=processed_data.get("special_requirements", ""),
                search_results=json.dumps(top_matches, indent=2)
            )
        
        # Get AI recommendations
        response = await llm.ainvoke([HumanMessage(content=prompt)])
        
        recommendations = {
            "ai_analysis": response.content,
            "top_matches": top_matches,
            "summary_stats": {
                "total_found": len(matches),
                "high_match": len([m for m in matches if m.get("match_score", 0) > 80]),
                "medium_match": len([m for m in matches if 60 <= m.get("match_score", 0) <= 80]),
                "low_match": len([m for m in matches if m.get("match_score", 0) < 60])
            }
        }
        
        print("✅ Generated comprehensive recommendations")
        return {"recommendations": recommendations}
    
    except Exception as e:
        print(f"❌ Error in generate_recommendations: {str(e)}")
        return {"error_message": f"Error generating recommendations: {str(e)}"}

async def format_final_output(state: WorkflowState) -> Dict:
    """Format final output for presentation"""
    try:
        # Check if recommendations exist
        if "recommendations" not in state:
            print("⚠️ No recommendations found in state, checking for alternatives...")
            # Try to format with what we have
            final_output = {
                "request_type": state.get("request_type", "unknown"),
                "timestamp": datetime.now().isoformat(),
                "status": "partial_success",
                "error": "Recommendations generation failed"
            }
            
            # Add matches if available
            if "matches" in state:
                final_output["matches"] = state["matches"][:10]
                final_output["total_matches"] = len(state["matches"])
            
            # Add any error messages
            if "error_message" in state:
                final_output["errors"] = state["error_message"]
            
            return {"final_output": final_output}
        
        recommendations = state["recommendations"]
        request_type = state["request_type"]
        
        print("📋 Formatting final output...")
        
        final_output = {
            "request_type": request_type,
            "timestamp": datetime.now().isoformat(),
            "recommendations": recommendations,
            "status": "success"
        }
        
        # Add any error messages if they exist
        if "error_message" in state:
            final_output["errors"] = state["error_message"]
            final_output["status"] = "partial_success"
        
        print("✅ Final output formatted successfully")
        return {"final_output": final_output}
    
    except Exception as e:
        print(f"❌ Error in format_final_output: {str(e)}")
        return {
            "final_output": {
                "status": "error",
                "message": str(e),
                "timestamp": datetime.now().isoformat()
            }
        }

# Create the LangGraph workflow
def create_job_matching_workflow():
    """Create the complete LangGraph workflow for job matching"""
    
    workflow = StateGraph(WorkflowState)
    
    # Add nodes
    workflow.add_node("process_input", process_user_input)
    workflow.add_node("search_linkedin", search_linkedin_profiles)
    workflow.add_node("enhance_data", enhance_search_data)
    workflow.add_node("calculate_matches", calculate_matches)
    workflow.add_node("generate_recommendations", generate_recommendations)
    workflow.add_node("format_output", format_final_output)
    
    # Add edges
    workflow.add_edge("process_input", "search_linkedin")
    workflow.add_edge("search_linkedin", "enhance_data")
    workflow.add_edge("enhance_data", "calculate_matches")
    workflow.add_edge("calculate_matches", "generate_recommendations")
    workflow.add_edge("generate_recommendations", "format_output")
    workflow.add_edge("format_output", END)
    
    # Set entry point
    workflow.set_entry_point("process_input")
    
    return workflow.compile()

# Initialize the workflow
job_matching_workflow = create_job_matching_workflow()

print("✅ LangGraph workflow created successfully!")
print("🔄 Workflow includes: Input Processing → LinkedIn Search → Data Enhancement → Match Calculation → AI Recommendations → Output Formatting")

## 8. Create Interactive User Interfaces

Building user-friendly interfaces for both job seekers and recruiters.

In [ ]:
class JobMatchingInterface:
    """Interactive interface for the job matching agent"""
    
    def __init__(self):
        self.current_mode = None
        self.results = None
    
    def create_job_seeker_form(self):
        """Create interactive form for job seekers"""
        
        # Form widgets
        name_widget = widgets.Text(
            value='',
            placeholder='Enter your full name',
            description='Name:',
            style={'description_width': 'initial'}
        )
        
        skills_widget = widgets.Textarea(
            value='',
            placeholder='Enter your skills separated by commas (e.g., Python, Machine Learning, AWS)',
            description='Skills:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(height='80px')
        )
        
        experience_widget = widgets.IntSlider(
            value=3,
            min=0,
            max=20,
            step=1,
            description='Experience (years):',
            style={'description_width': 'initial'}
        )
        
        education_widget = widgets.Dropdown(
            options=['High School', 'Associates Degree', 'Bachelors Degree', 'Masters Degree', 'PhD', 'Other'],
            value='Bachelors Degree',
            description='Education:',
            style={'description_width': 'initial'}
        )
        
        location_widget = widgets.Text(
            value='',
            placeholder='Enter your location (e.g., San Francisco, CA)',
            description='Location:',
            style={'description_width': 'initial'}
        )
        
        salary_widget = widgets.Text(
            value='',
            placeholder='Enter desired salary range (e.g., $80,000 - $120,000)',
            description='Salary Range:',
            style={'description_width': 'initial'}
        )
        
        job_prefs_widget = widgets.Textarea(
            value='',
            placeholder='Enter preferred job titles separated by commas',
            description='Job Preferences:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(height='60px')
        )
        
        industry_prefs_widget = widgets.Textarea(
            value='',
            placeholder='Enter preferred industries separated by commas',
            description='Industry Preferences:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(height='60px')
        )
        
        submit_button = widgets.Button(
            description='🔍 Find Jobs',
            button_style='primary',
            layout=widgets.Layout(width='200px', height='40px')
        )
        
        output = widgets.Output()
        
        def on_submit_clicked(b):
            with output:
                clear_output()
                try:
                    # Collect form data
                    job_seeker_data = {
                        "name": name_widget.value.strip(),
                        "skills": [skill.strip() for skill in skills_widget.value.split(',') if skill.strip()],
                        "experience_years": experience_widget.value,
                        "education": education_widget.value,
                        "location": location_widget.value.strip(),
                        "desired_salary": salary_widget.value.strip(),
                        "job_preferences": [pref.strip() for pref in job_prefs_widget.value.split(',') if pref.strip()],
                        "industry_preferences": [ind.strip() for ind in industry_prefs_widget.value.split(',') if ind.strip()],
                        "additional_requirements": ""
                    }
                    
                    # Validate required fields
                    if not job_seeker_data["name"]:
                        print("❌ Please enter your name")
                        return
                    if not job_seeker_data["skills"]:
                        print("❌ Please enter at least one skill")
                        return
                    if not job_seeker_data["location"]:
                        print("❌ Please enter your location")
                        return
                    
                    print("🚀 Starting job search for", job_seeker_data["name"])
                    print("📋 Skills:", ", ".join(job_seeker_data["skills"]))
                    print("📍 Location:", job_seeker_data["location"])
                    print("💼 Experience:", job_seeker_data["experience_years"], "years")
                    print("\n" + "="*50)
                    
                    # Store for processing
                    self.job_seeker_data = job_seeker_data
                    self.current_mode = "job_seeker"
                    
                except Exception as e:
                    print(f"❌ Error: {str(e)}")
        
        submit_button.on_click(on_submit_clicked)
        
        # Layout
        form_items = [
            widgets.HTML("<h3>🔍 Job Seeker Profile</h3>"),
            name_widget,
            skills_widget,
            experience_widget,
            education_widget,
            location_widget,
            salary_widget,
            job_prefs_widget,
            industry_prefs_widget,
            submit_button,
            output
        ]
        
        return widgets.VBox(form_items, layout=widgets.Layout(padding='20px'))
    
    def create_recruiter_form(self):
        """Create interactive form for recruiters"""
        
        # Form widgets
        job_title_widget = widgets.Text(
            value='',
            placeholder='Enter job title (e.g., Senior Software Engineer)',
            description='Job Title:',
            style={'description_width': 'initial'}
        )
        
        company_widget = widgets.Text(
            value='',
            placeholder='Enter company name',
            description='Company:',
            style={'description_width': 'initial'}
        )
        
        location_widget = widgets.Text(
            value='',
            placeholder='Enter job location (e.g., New York, NY or Remote)',
            description='Location:',
            style={'description_width': 'initial'}
        )
        
        experience_widget = widgets.Text(
            value='',
            placeholder='Enter experience requirements (e.g., 3-5 years)',
            description='Experience Required:',
            style={'description_width': 'initial'}
        )
        
        required_skills_widget = widgets.Textarea(
            value='',
            placeholder='Enter required skills separated by commas',
            description='Required Skills:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(height='80px')
        )
        
        preferred_skills_widget = widgets.Textarea(
            value='',
            placeholder='Enter nice-to-have skills separated by commas',
            description='Preferred Skills:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(height='80px')
        )
        
        education_widget = widgets.Text(
            value='',
            placeholder='Enter education requirements (e.g., Bachelors in Computer Science)',
            description='Education:',
            style={'description_width': 'initial'}
        )
        
        salary_widget = widgets.Text(
            value='',
            placeholder='Enter salary range (e.g., $100,000 - $150,000)',
            description='Salary Range:',
            style={'description_width': 'initial'}
        )
        
        culture_widget = widgets.Textarea(
            value='',
            placeholder='Describe company culture and work environment',
            description='Company Culture:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(height='80px')
        )
        
        submit_button = widgets.Button(
            description='👥 Find Candidates',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px')
        )
        
        output = widgets.Output()
        
        def on_submit_clicked(b):
            with output:
                clear_output()
                try:
                    # Collect form data
                    job_requirements = {
                        "job_title": job_title_widget.value.strip(),
                        "company_name": company_widget.value.strip(),
                        "job_location": location_widget.value.strip(),
                        "experience_required": experience_widget.value.strip(),
                        "required_skills": [skill.strip() for skill in required_skills_widget.value.split(',') if skill.strip()],
                        "preferred_skills": [skill.strip() for skill in preferred_skills_widget.value.split(',') if skill.strip()],
                        "education_requirements": education_widget.value.strip(),
                        "salary_range": salary_widget.value.strip(),
                        "company_culture": culture_widget.value.strip(),
                        "special_requirements": ""
                    }
                    
                    # Validate required fields
                    if not job_requirements["job_title"]:
                        print("❌ Please enter the job title")
                        return
                    if not job_requirements["required_skills"]:
                        print("❌ Please enter at least one required skill")
                        return
                    if not job_requirements["job_location"]:
                        print("❌ Please enter the job location")
                        return
                    
                    print("🚀 Starting candidate search for", job_requirements["job_title"])
                    print("🏢 Company:", job_requirements["company_name"])
                    print("📍 Location:", job_requirements["job_location"])
                    print("🛠️ Required Skills:", ", ".join(job_requirements["required_skills"]))
                    print("\n" + "="*50)
                    
                    # Store for processing
                    self.job_requirements = job_requirements
                    self.current_mode = "recruiter"
                    
                except Exception as e:
                    print(f"❌ Error: {str(e)}")
        
        submit_button.on_click(on_submit_clicked)
        
        # Layout
        form_items = [
            widgets.HTML("<h3>👥 Job Requirements</h3>"),
            job_title_widget,
            company_widget,
            location_widget,
            experience_widget,
            required_skills_widget,
            preferred_skills_widget,
            education_widget,
            salary_widget,
            culture_widget,
            submit_button,
            output
        ]
        
        return widgets.VBox(form_items, layout=widgets.Layout(padding='20px'))
    
    def display_mode_selector(self):
        """Display mode selection interface"""
        
        mode_selector = widgets.ToggleButtons(
            options=['Job Seeker', 'Recruiter'],
            description='I am a:',
            button_style='info',
            style={'description_width': 'initial'}
        )
        
        display_area = widgets.Output()
        
        def on_mode_change(change):
            with display_area:
                clear_output()
                if change['new'] == 'Job Seeker':
                    form = self.create_job_seeker_form()
                else:
                    form = self.create_recruiter_form()
                display(form)
        
        mode_selector.observe(on_mode_change, names='value')
        
        # Display initial form
        with display_area:
            form = self.create_job_seeker_form()  # Default to job seeker
            display(form)
        
        return widgets.VBox([
            widgets.HTML("<h2>🤖 AI Job Matching Agent</h2>"),
            widgets.HTML("<p>Select your role to get started:</p>"),
            mode_selector,
            display_area
        ])

# Initialize interface
interface = JobMatchingInterface()

print("✅ Interactive interfaces created successfully!")
print("🎨 Interfaces include: Job Seeker Form and Recruiter Form with validation")

## 9. Implement Job Seeker Assistant

Complete implementation of the job seeker assistant with personalized recommendations.

In [ ]:
async def process_job_seeker_request(job_seeker_data: Dict) -> Dict:
    """Process job seeker request and return job recommendations"""
    
    try:
        print("🔄 Processing job seeker request...")
        
        # Run the workflow
        result = await job_matching_workflow.ainvoke({
            "request_type": "job_seeker",
            "user_input": job_seeker_data
        })
        
        return result["final_output"]
    
    except Exception as e:
        print(f"❌ Error processing job seeker request: {str(e)}")
        return {"status": "error", "message": str(e)}

def display_job_recommendations(results: Dict):
    """Display job recommendations with rich formatting"""
    
    if results.get("status") == "error":
        display(HTML(f"""
        <div style="border: 2px solid #FF6B6B; padding: 20px; border-radius: 10px; background-color: #FFE8E8;">
            <h3 style="color: #FF6B6B;">❌ Error</h3>
            <p>{results.get("message", "Unknown error occurred")}</p>
        </div>
        """))
        return
    
    # Extract recommendations
    recommendations = results.get("recommendations", {})
    top_matches = recommendations.get("top_matches", [])
    ai_analysis = recommendations.get("ai_analysis", "")
    summary_stats = recommendations.get("summary_stats", {})
    
    # Display summary statistics
    display(HTML(f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; margin-bottom: 20px;">
        <h2 style="margin: 0; color: white;">📊 Job Search Results Summary</h2>
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-top: 15px;">
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; text-align: center;">
                <h3 style="margin: 0; color: white;">{summary_stats.get('total_found', 0)}</h3>
                <p style="margin: 5px 0 0 0;">Total Opportunities</p>
            </div>
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; text-align: center;">
                <h3 style="margin: 0; color: white;">{summary_stats.get('high_match', 0)}</h3>
                <p style="margin: 5px 0 0 0;">High Match (80%+)</p>
            </div>
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; text-align: center;">
                <h3 style="margin: 0; color: white;">{summary_stats.get('medium_match', 0)}</h3>
                <p style="margin: 5px 0 0 0;">Medium Match (60-80%)</p>
            </div>
        </div>
    </div>
    """))
    
    # Display AI analysis
    if ai_analysis:
        display(HTML(f"""
        <div style="border: 2px solid #4ECDC4; padding: 20px; border-radius: 10px; background-color: #F0FDFF; margin-bottom: 20px;">
            <h3 style="color: #4ECDC4; margin-top: 0;">🧠 AI Career Advisor Analysis</h3>
            <div style="white-space: pre-wrap; line-height: 1.6;">{ai_analysis}</div>
        </div>
        """))
    
    # Display top job matches
    if top_matches:
        display(HTML("<h3 style='color: #2E86AB; margin-top: 30px;'>🎯 Top Job Recommendations</h3>"))
        
        for i, job in enumerate(top_matches[:8], 1):
            match_score = job.get("match_score", 0)
            match_details = job.get("match_details", {})
            
            # Determine match color
            if match_score >= 80:
                score_color = "#2ECC71"
                match_level = "Excellent Match"
            elif match_score >= 60:
                score_color = "#F39C12"
                match_level = "Good Match"
            else:
                score_color = "#E74C3C"
                match_level = "Potential Match"
            
            # Format skills
            required_skills = job.get("required_skills", [])
            skills_display = ", ".join(required_skills[:5]) if required_skills else "Not specified"
            
            display(HTML(f"""
            <div style="border: 1px solid #ddd; padding: 20px; margin: 15px 0; border-radius: 12px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
                <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px;">
                    <h3 style="color: #2E86AB; margin: 0;">#{i} {job.get('title', 'Unknown Position')}</h3>
                    <div style="background: {score_color}; color: white; padding: 8px 16px; border-radius: 20px; font-weight: bold;">
                        {match_score}% - {match_level}
                    </div>
                </div>
                
                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 15px;">
                    <div>
                        <p><strong>🏢 Company:</strong> {job.get('company', 'Unknown Company')}</p>
                        <p><strong>📍 Location:</strong> {job.get('location', 'Unknown Location')}</p>
                        <p><strong>💰 Salary:</strong> {job.get('salary_range', 'Not specified')}</p>
                    </div>
                    <div>
                        <p><strong>🛠️ Required Skills:</strong> {skills_display}</p>
                        <p><strong>📅 Experience:</strong> {job.get('experience_required', 'Not specified')}</p>
                        <p><strong>🔗 Apply:</strong> <a href="{job.get('url', '#')}" target="_blank" style="color: #2E86AB;">View Job Posting</a></p>
                    </div>
                </div>
                
                {f'''
                <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin-top: 15px;">
                    <h4 style="margin: 0 0 10px 0; color: #495057;">📈 Match Breakdown:</h4>
                    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(120px, 1fr)); gap: 10px;">
                        <div>Skills: {match_details.get('skills_score', 0)}%</div>
                        <div>Experience: {match_details.get('experience_score', 0)}%</div>
                        <div>Location: {match_details.get('location_score', 0)}%</div>
                        <div>Education: {match_details.get('education_score', 0)}%</div>
                    </div>
                </div>
                ''' if match_details else ''}
            </div>
            """))

def create_job_seeker_dashboard():
    """Create complete job seeker dashboard"""
    
    # Processing button
    process_button = widgets.Button(
        description='🚀 Start Job Search',
        button_style='primary',
        layout=widgets.Layout(width='200px', height='50px', margin='20px 0')
    )
    
    results_output = widgets.Output()
    
    async def process_and_display():
        """Process job seeker request and display results"""
        try:
            if not hasattr(interface, 'job_seeker_data'):
                with results_output:
                    clear_output()
                    print("❌ Please fill out the job seeker form first!")
                return
            
            with results_output:
                clear_output()
                print("🔄 Starting comprehensive job search...")
                print("📡 This may take 30-60 seconds to search LinkedIn and analyze matches...")
                
                # Process the request
                results = await process_job_seeker_request(interface.job_seeker_data)
                
                # Display results
                display_job_recommendations(results)
                
        except Exception as e:
            with results_output:
                clear_output()
                print(f"❌ Error during job search: {str(e)}")
                print("🔧 Please check your input and try again.")
    
    def on_process_clicked(b):
        # Run async function in thread for Jupyter compatibility
        import threading
        thread = threading.Thread(target=lambda: asyncio.run(process_and_display()))
        thread.start()
    
    process_button.on_click(on_process_clicked)
    
    return widgets.VBox([
        widgets.HTML("<hr style='margin: 30px 0;'>"),
        widgets.HTML("<h3>🚀 Execute Job Search</h3>"),
        widgets.HTML("<p>Click the button below to start your AI-powered job search after filling out the form above.</p>"),
        process_button,
        results_output
    ])

print("✅ Job Seeker Assistant implemented successfully!")
print("🎯 Features include: Request processing, job recommendations display, and comprehensive dashboard")

## 10. Implement Recruiter Assistant

Complete implementation of the recruiter assistant for candidate sourcing and ranking.

In [ ]:
async def process_recruiter_request(job_requirements: Dict) -> Dict:
    """Process recruiter request and return candidate recommendations"""
    
    try:
        print("🔄 Processing recruiter request...")
        
        # Run the workflow
        result = await job_matching_workflow.ainvoke({
            "request_type": "recruiter",
            "user_input": job_requirements
        })
        
        return result["final_output"]
    
    except Exception as e:
        print(f"❌ Error processing recruiter request: {str(e)}")
        return {"status": "error", "message": str(e)}

def display_candidate_recommendations(results: Dict):
    """Display candidate recommendations with rich formatting"""
    
    if results.get("status") == "error":
        display(HTML(f"""
        <div style="border: 2px solid #FF6B6B; padding: 20px; border-radius: 10px; background-color: #FFE8E8;">
            <h3 style="color: #FF6B6B;">❌ Error</h3>
            <p>{results.get("message", "Unknown error occurred")}</p>
        </div>
        """))
        return
    
    # Extract recommendations
    recommendations = results.get("recommendations", {})
    top_matches = recommendations.get("top_matches", [])
    ai_analysis = recommendations.get("ai_analysis", "")
    summary_stats = recommendations.get("summary_stats", {})
    
    # Display summary statistics
    display(HTML(f"""
    <div style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); padding: 20px; border-radius: 15px; color: white; margin-bottom: 20px;">
        <h2 style="margin: 0; color: white;">📊 Candidate Search Results Summary</h2>
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-top: 15px;">
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; text-align: center;">
                <h3 style="margin: 0; color: white;">{summary_stats.get('total_found', 0)}</h3>
                <p style="margin: 5px 0 0 0;">Total Candidates</p>
            </div>
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; text-align: center;">
                <h3 style="margin: 0; color: white;">{summary_stats.get('high_match', 0)}</h3>
                <p style="margin: 5px 0 0 0;">Excellent Fit (80%+)</p>
            </div>
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; text-align: center;">
                <h3 style="margin: 0; color: white;">{summary_stats.get('medium_match', 0)}</h3>
                <p style="margin: 5px 0 0 0;">Good Fit (60-80%)</p>
            </div>
        </div>
    </div>
    """))
    
    # Display AI analysis
    if ai_analysis:
        display(HTML(f"""
        <div style="border: 2px solid #2ECC71; padding: 20px; border-radius: 10px; background-color: #F0FFF0; margin-bottom: 20px;">
            <h3 style="color: #2ECC71; margin-top: 0;">🧠 AI Recruitment Specialist Analysis</h3>
            <div style="white-space: pre-wrap; line-height: 1.6;">{ai_analysis}</div>
        </div>
        """))
    
    # Display top candidate matches
    if top_matches:
        display(HTML("<h3 style='color: #27AE60; margin-top: 30px;'>🎯 Top Candidate Recommendations</h3>"))
        
        for i, candidate in enumerate(top_matches[:8], 1):
            match_score = candidate.get("match_score", 0)
            match_details = candidate.get("match_details", {})
            
            # Determine match color and tier
            if match_score >= 80:
                score_color = "#2ECC71"
                match_level = "Excellent Fit"
                tier_icon = "⭐"
            elif match_score >= 60:
                score_color = "#F39C12"
                match_level = "Good Fit"
                tier_icon = "👍"
            else:
                score_color = "#E74C3C"
                match_level = "Potential Fit"
                tier_icon = "🤔"
            
            # Format candidate skills
            candidate_skills = candidate.get("skills", [])
            skills_display = ", ".join(candidate_skills[:5]) if candidate_skills else "Not specified"
            
            # Extract basic info
            name = candidate.get("name", "Unknown Candidate")
            current_position = candidate.get("current_position", "Unknown Position")
            company = candidate.get("company", "Unknown Company")
            location = candidate.get("location", "Unknown Location")
            experience = candidate.get("experience", "Unknown Experience")
            education = candidate.get("education", "Unknown Education")
            linkedin_url = candidate.get("linkedin_url", "#")
            
            display(HTML(f"""
            <div style="border: 1px solid #ddd; padding: 20px; margin: 15px 0; border-radius: 12px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
                <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px;">
                    <h3 style="color: #27AE60; margin: 0;">{tier_icon} #{i} {name}</h3>
                    <div style="background: {score_color}; color: white; padding: 8px 16px; border-radius: 20px; font-weight: bold;">
                        {match_score}% - {match_level}
                    </div>
                </div>
                
                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 15px;">
                    <div>
                        <p><strong>💼 Current Role:</strong> {current_position}</p>
                        <p><strong>🏢 Company:</strong> {company}</p>
                        <p><strong>📍 Location:</strong> {location}</p>
                        <p><strong>🔗 LinkedIn:</strong> <a href="{linkedin_url}" target="_blank" style="color: #27AE60;">View Profile</a></p>
                    </div>
                    <div>
                        <p><strong>🛠️ Skills:</strong> {skills_display}</p>
                        <p><strong>📅 Experience:</strong> {experience}</p>
                        <p><strong>🎓 Education:</strong> {education}</p>
                        <p><strong>📧 Status:</strong> <span style="color: #27AE60;">Ready to Connect</span></p>
                    </div>
                </div>
                
                {f'''
                <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin-top: 15px;">
                    <h4 style="margin: 0 0 10px 0; color: #495057;">📈 Fit Analysis:</h4>
                    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(120px, 1fr)); gap: 10px; margin-bottom: 10px;">
                        <div>Skills: {match_details.get('skills_score', 0)}%</div>
                        <div>Experience: {match_details.get('experience_score', 0)}%</div>
                        <div>Location: {match_details.get('location_score', 0)}%</div>
                        <div>Education: {match_details.get('education_score', 0)}%</div>
                    </div>
                    <div style="margin-top: 10px;">
                        <strong>💪 Strengths:</strong> {', '.join(match_details.get('strengths', ['Profile analysis available']))}
                    </div>
                    {f'<div style="margin-top: 5px;"><strong>📋 Development Areas:</strong> {", ".join(match_details.get("gaps", ["None identified"]))}</div>' if match_details.get('gaps') else ''}
                </div>
                ''' if match_details else ''}
                
                <div style="margin-top: 15px; padding: 10px; background: linear-gradient(90deg, #f8f9fa 0%, #e9ecef 100%); border-radius: 8px;">
                    <strong>🎯 Recruitment Recommendation:</strong> 
                    {f"Priority candidate for immediate interview" if match_score >= 80 else 
                     f"Strong candidate - schedule phone screening" if match_score >= 60 else 
                     f"Consider for future opportunities or upskilling"}
                </div>
            </div>
            """))

def create_recruiter_dashboard():
    """Create complete recruiter dashboard"""
    
    # Processing button
    process_button = widgets.Button(
        description='🔍 Search Candidates',
        button_style='success',
        layout=widgets.Layout(width='200px', height='50px', margin='20px 0')
    )
    
    results_output = widgets.Output()
    
    async def process_and_display():
        """Process recruiter request and display results"""
        try:
            if not hasattr(interface, 'job_requirements'):
                with results_output:
                    clear_output()
                    print("❌ Please fill out the job requirements form first!")
                return
            
            with results_output:
                clear_output()
                print("🔄 Starting comprehensive candidate search...")
                print("📡 This may take 30-60 seconds to search LinkedIn and analyze candidates...")
                
                # Process the request
                results = await process_recruiter_request(interface.job_requirements)
                
                # Display results
                display_candidate_recommendations(results)
                
        except Exception as e:
            with results_output:
                clear_output()
                print(f"❌ Error during candidate search: {str(e)}")
                print("🔧 Please check your input and try again.")
    
    def on_process_clicked(b):
        # Run async function in thread for Jupyter compatibility
        import threading
        thread = threading.Thread(target=lambda: asyncio.run(process_and_display()))
        thread.start()
    
    process_button.on_click(on_process_clicked)
    
    return widgets.VBox([
        widgets.HTML("<hr style='margin: 30px 0;'>"),
        widgets.HTML("<h3>🔍 Execute Candidate Search</h3>"),
        widgets.HTML("<p>Click the button below to start your AI-powered candidate search after filling out the form above.</p>"),
        process_button,
        results_output
    ])

print("✅ Recruiter Assistant implemented successfully!")
print("👥 Features include: Request processing, candidate recommendations display, and comprehensive dashboard")

## 11. Add Result Visualization and Analytics

Implementing rich visualizations and analytics for better insights and decision making.

In [ ]:
class JobMatchingAnalytics:
    """Analytics and visualization for job matching results"""
    
    def __init__(self):
        self.color_palette = {
            'primary': '#2E86AB',
            'secondary': '#A23B72',
            'success': '#2ECC71',
            'warning': '#F39C12',
            'danger': '#E74C3C',
            'info': '#3498DB'
        }
    
    def create_match_score_distribution(self, results: List[Dict], title: str = "Match Score Distribution"):
        """Create match score distribution visualization"""
        
        scores = [result.get("match_score", 0) for result in results]
        
        # Create histogram
        fig = go.Figure(data=[
            go.Histogram(
                x=scores,
                nbinsx=20,
                marker_color=self.color_palette['primary'],
                opacity=0.7,
                name='Match Scores'
            )
        ])
        
        fig.update_layout(
            title={
                'text': title,
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            xaxis_title="Match Score (%)",
            yaxis_title="Number of Results",
            template="plotly_white",
            showlegend=False,
            height=400
        )
        
        # Add average line
        avg_score = np.mean(scores) if scores else 0
        fig.add_vline(
            x=avg_score, 
            line_dash="dash", 
            line_color=self.color_palette['danger'],
            annotation_text=f"Average: {avg_score:.1f}%"
        )
        
        fig.show()
        
        return fig
    
    def create_skills_demand_analysis(self, results: List[Dict], result_type: str = "jobs"):
        """Create skills demand/supply analysis"""
        
        all_skills = []
        
        if result_type == "jobs":
            for result in results:
                skills = result.get("required_skills", [])
                all_skills.extend(skills)
        else:  # candidates
            for result in results:
                skills = result.get("skills", [])
                all_skills.extend(skills)
        
        if not all_skills:
            print("No skills data available for visualization")
            return None
        
        # Count skills
        skill_counts = pd.Series(all_skills).value_counts().head(15)
        
        # Create bar chart
        fig = go.Figure(data=[
            go.Bar(
                x=skill_counts.values,
                y=skill_counts.index,
                orientation='h',
                marker_color=self.color_palette['info'],
                text=skill_counts.values,
                textposition='outside'
            )
        ])
        
        title_text = f"Top 15 Most In-Demand Skills" if result_type == "jobs" else f"Top 15 Most Common Skills in Candidates"
        
        fig.update_layout(
            title={
                'text': title_text,
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            xaxis_title="Frequency",
            yaxis_title="Skills",
            template="plotly_white",
            height=600,
            showlegend=False
        )
        
        fig.show()
        
        return fig
    
    def create_location_distribution(self, results: List[Dict], title: str = "Location Distribution"):
        """Create location distribution pie chart"""
        
        locations = [result.get("location", "Unknown") for result in results]
        
        # Clean and group locations
        location_counts = pd.Series(locations).value_counts().head(10)
        
        # Create pie chart
        fig = go.Figure(data=[
            go.Pie(
                labels=location_counts.index,
                values=location_counts.values,
                hole=0.3,
                marker_colors=px.colors.qualitative.Set3
            )
        ])
        
        fig.update_layout(
            title={
                'text': title,
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            template="plotly_white",
            height=500
        )
        
        fig.show()
        
        return fig
    
    def create_experience_analysis(self, results: List[Dict], requirements: Dict = None):
        """Create experience level analysis"""
        
        experience_data = []
        
        for result in results:
            exp_text = result.get("experience", "") or result.get("experience_required", "")
            years = self.extract_years_from_text(exp_text)
            experience_data.append({
                'years': years,
                'match_score': result.get('match_score', 0),
                'title': result.get('title', '') or result.get('name', '')
            })
        
        df = pd.DataFrame(experience_data)
        
        if df.empty:
            print("No experience data available for visualization")
            return None
        
        # Create scatter plot
        fig = go.Figure(data=[
            go.Scatter(
                x=df['years'],
                y=df['match_score'],
                mode='markers',
                marker=dict(
                    size=10,
                    color=df['match_score'],
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title="Match Score %")
                ),
                text=df['title'],
                hovertemplate='<b>%{text}</b><br>Experience: %{x} years<br>Match Score: %{y}%<extra></extra>'
            )
        ])
        
        fig.update_layout(
            title={
                'text': 'Experience vs Match Score Analysis',
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            xaxis_title="Years of Experience",
            yaxis_title="Match Score (%)",
            template="plotly_white",
            height=500
        )
        
        # Add requirement line if available
        if requirements and "experience_required" in requirements:
            req_years = self.extract_years_from_text(requirements["experience_required"])
            if req_years > 0:
                fig.add_vline(
                    x=req_years,
                    line_dash="dash",
                    line_color=self.color_palette['danger'],
                    annotation_text=f"Required: {req_years} years"
                )
        
        fig.show()
        
        return fig
    
    def extract_years_from_text(self, text: str) -> int:
        """Extract years from experience text"""
        import re
        
        if not text:
            return 0
        
        patterns = [
            r'(\d+)\+?\s*years?',
            r'(\d+)-\d+\s*years?',
            r'(\d+)\s*yrs?',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                return int(match.group(1))
        
        return 0
    
    def create_comprehensive_dashboard(self, results: List[Dict], request_type: str, requirements: Dict = None):
        """Create comprehensive analytics dashboard"""
        
        print(f"📊 Generating comprehensive analytics dashboard for {request_type}...")
        
        # Match Score Distribution
        self.create_match_score_distribution(
            results, 
            f"Match Score Distribution - {request_type.title()}"
        )
        
        # Skills Analysis
        self.create_skills_demand_analysis(
            results, 
            "jobs" if request_type == "job_seeker" else "candidates"
        )
        
        # Location Distribution
        self.create_location_distribution(
            results,
            f"Location Distribution - {request_type.title()}"
        )
        
        # Experience Analysis
        self.create_experience_analysis(results, requirements)
        
        # Summary Statistics
        self.display_summary_statistics(results, request_type)
    
    def display_summary_statistics(self, results: List[Dict], request_type: str):
        """Display comprehensive summary statistics"""
        
        if not results:
            print("No results available for analysis")
            return
        
        scores = [r.get("match_score", 0) for r in results]
        
        stats = {
            "Total Results": len(results),
            "Average Match Score": f"{np.mean(scores):.1f}%",
            "Highest Match Score": f"{max(scores):.1f}%",
            "Lowest Match Score": f"{min(scores):.1f}%",
            "Standard Deviation": f"{np.std(scores):.1f}%",
            "Excellent Matches (80%+)": len([s for s in scores if s >= 80]),
            "Good Matches (60-80%)": len([s for s in scores if 60 <= s < 80]),
            "Fair Matches (<60%)": len([s for s in scores if s < 60])
        }
        
        # Create HTML table
        html_content = f"""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; margin: 20px 0;">
            <h3 style="margin: 0 0 15px 0; color: white;">📊 Analytics Summary - {request_type.title()}</h3>
            <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 15px;">
        """
        
        for key, value in stats.items():
            html_content += f"""
                <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px;">
                    <strong>{key}:</strong><br>
                    <span style="font-size: 1.2em;">{value}</span>
                </div>
            """
        
        html_content += """
            </div>
        </div>
        """
        
        display(HTML(html_content))

# Initialize analytics
analytics = JobMatchingAnalytics()

# Enhanced display functions with analytics
def display_job_recommendations_with_analytics(results: Dict):
    """Display job recommendations with analytics"""
    display_job_recommendations(results)
    
    # Add analytics if results are available
    recommendations = results.get("recommendations", {})
    top_matches = recommendations.get("top_matches", [])
    
    if top_matches:
        print("\n" + "="*50)
        print("📊 GENERATING ANALYTICS DASHBOARD")
        print("="*50)
        
        analytics.create_comprehensive_dashboard(
            top_matches, 
            "job_seeker",
            None  # Can pass job seeker requirements here if needed
        )

def display_candidate_recommendations_with_analytics(results: Dict, requirements: Dict = None):
    """Display candidate recommendations with analytics"""
    display_candidate_recommendations(results)
    
    # Add analytics if results are available
    recommendations = results.get("recommendations", {})
    top_matches = recommendations.get("top_matches", [])
    
    if top_matches:
        print("\n" + "="*50)
        print("📊 GENERATING ANALYTICS DASHBOARD")
        print("="*50)
        
        analytics.create_comprehensive_dashboard(
            top_matches, 
            "recruiter",
            requirements
        )

print("✅ Analytics and visualization system implemented successfully!")
print("📊 Features include: Match score distributions, skills analysis, location mapping, experience analysis, and comprehensive dashboards")

In [ ]:
class JobMatchingAnalytics:
    """Analytics and visualization system for job matching results"""
    
    def __init__(self):
        self.color_palette = {
            'primary': '#3498db',
            'success': '#2ecc71', 
            'warning': '#f39c12',
            'danger': '#e74c3c',
            'info': '#9b59b6'
        }
    
    def extract_years_from_text(self, text: str) -> int:
        """Extract years of experience from text"""
        import re
        
        if not text:
            return 0
        
        # Look for patterns like "5 years", "3-5 years", "5+ years"
        patterns = [
            r'(\d+)\+?\s*years?',
            r'(\d+)-\d+\s*years?',
            r'(\d+)\s*yr',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                return int(match.group(1))
        
        return 0
    
    def create_match_score_distribution(self, results: List[Dict], title: str = "Match Score Distribution"):
        """Create match score distribution visualization"""
        
        scores = [r.get("match_score", 0) for r in results if "match_score" in r]
        
        if not scores:
            print("No match scores available for visualization")
            return None
        
        # Create histogram
        fig = go.Figure(data=[
            go.Histogram(
                x=scores,
                nbinsx=10,
                marker_color=self.color_palette['primary'],
                opacity=0.7
            )
        ])
        
        fig.update_layout(
            title={
                'text': title,
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            xaxis_title="Match Score (%)",
            yaxis_title="Number of Results",
            template="plotly_white",
            showlegend=False,
            height=400
        )
        
        # Add average line
        avg_score = np.mean(scores) if scores else 0
        fig.add_vline(
            x=avg_score, 
            line_dash="dash", 
            line_color=self.color_palette['danger'],
            annotation_text=f"Average: {avg_score:.1f}%"
        )
        
        fig.show()
        return fig
    
    def create_skills_demand_analysis(self, results: List[Dict], result_type: str = "jobs"):
        """Create skills demand/supply analysis"""
        
        all_skills = []
        
        if result_type == "jobs":
            for result in results:
                skills = result.get("required_skills", [])
                all_skills.extend(skills)
        else:  # candidates
            for result in results:
                skills = result.get("skills", [])
                all_skills.extend(skills)
        
        if not all_skills:
            print("No skills data available for analysis")
            return None
        
        # Count skill frequency
        skill_counts = {}
        for skill in all_skills:
            if skill:
                skill_counts[skill] = skill_counts.get(skill, 0) + 1
        
        # Sort by frequency and take top 15
        top_skills = sorted(skill_counts.items(), key=lambda x: x[1], reverse=True)[:15]
        
        if not top_skills:
            print("No skills data available for visualization")
            return None
        
        skills, counts = zip(*top_skills)
        
        # Create bar chart
        fig = go.Figure(data=[
            go.Bar(
                x=list(skills),
                y=list(counts),
                marker_color=self.color_palette['success'],
                text=list(counts),
                textposition='auto'
            )
        ])
        
        fig.update_layout(
            title={
                'text': f'Most In-Demand Skills ({result_type.title()})',
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            xaxis_title="Skills",
            yaxis_title="Frequency",
            template="plotly_white",
            height=500
        )
        
        fig.update_xaxes(tickangle=45)
        fig.show()
        return fig
    
    def create_location_analysis(self, results: List[Dict], result_type: str = "jobs"):
        """Create location-based analysis"""
        
        locations = []
        
        for result in results:
            location = result.get("location", "")
            if location:
                locations.append(location)
        
        if not locations:
            print("No location data available for analysis")
            return None
        
        # Count location frequency
        location_counts = {}
        for location in locations:
            location_counts[location] = location_counts.get(location, 0) + 1
        
        # Sort by frequency and take top 10
        top_locations = sorted(location_counts.items(), key=lambda x: x[1], reverse=True)[:10]
        
        if not top_locations:
            print("No location data available for visualization")
            return None
        
        locations, counts = zip(*top_locations)
        
        # Create pie chart
        fig = go.Figure(data=[
            go.Pie(
                labels=list(locations),
                values=list(counts),
                hole=0.4,
                marker_colors=[self.color_palette[color] for color in ['primary', 'success', 'warning', 'danger', 'info'] * 2]
            )
        ])
        
        fig.update_layout(
            title={
                'text': f'Geographic Distribution ({result_type.title()})',
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            height=500
        )
        
        fig.show()
        return fig
    
    def create_experience_analysis(self, results: List[Dict]):
        """Create experience vs match score analysis"""
        
        data_points = []
        
        for result in results:
            exp_text = result.get("experience", "")
            years = self.extract_years_from_text(exp_text) if exp_text else 0
            score = result.get("match_score", 0)
            title = result.get("title", "Unknown")
            
            if years > 0:  # Only include if we could extract experience
                data_points.append({
                    'years': years,
                    'match_score': score,
                    'title': title
                })
        
        if not data_points:
            print("No experience data available for analysis")
            return None
        
        import pandas as pd
        df = pd.DataFrame(data_points)
        
        if df.empty:
            print("No experience data available for visualization")
            return None
        
        # Create scatter plot
        fig = go.Figure(data=[
            go.Scatter(
                x=df['years'],
                y=df['match_score'],
                mode='markers',
                marker=dict(
                    size=10,
                    color=df['match_score'],
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title="Match Score %")
                ),
                text=df['title'],
                hovertemplate='<b>%{text}</b><br>Experience: %{x} years<br>Match Score: %{y}%<extra></extra>'
            )
        ])
        
        fig.update_layout(
            title={
                'text': 'Experience vs Match Score Analysis',
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            xaxis_title="Years of Experience",
            yaxis_title="Match Score (%)",
            template="plotly_white",
            height=500
        )
        
        fig.show()
        return fig
    
    def display_summary_statistics(self, results: List[Dict], request_type: str):
        """Display comprehensive summary statistics"""
        
        if not results:
            print("No results available for analysis")
            return
        
        scores = [r.get("match_score", 0) for r in results]
        
        stats = {
            "Total Results": len(results),
            "Average Match Score": f"{np.mean(scores):.1f}%",
            "Highest Match Score": f"{max(scores):.1f}%", 
            "Lowest Match Score": f"{min(scores):.1f}%",
            "Standard Deviation": f"{np.std(scores):.1f}%",
            "Excellent Matches (80%+)": len([s for s in scores if s >= 80]),
            "Good Matches (60-80%)": len([s for s in scores if 60 <= s < 80]),
            "Fair Matches (<60%)": len([s for s in scores if s < 60])
        }
        
        # Create HTML table
        html_content = f"""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; margin: 20px 0;">
            <h3 style="margin: 0 0 15px 0; color: white;">📊 Analytics Summary - {request_type.title()}</h3>
            <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 15px;">
        """
        
        for key, value in stats.items():
            html_content += f"""
                <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px;">
                    <strong>{key}:</strong><br>
                    <span style="font-size: 1.3em; font-weight: bold;">{value}</span>
                </div>
            """
        
        html_content += """
            </div>
        </div>
        """
        
        display(HTML(html_content))
    
    def create_comprehensive_dashboard(self, results: List[Dict], request_type: str, requirements: Dict = None):
        """Create a comprehensive analytics dashboard"""
        
        print("📊 COMPREHENSIVE ANALYTICS DASHBOARD")
        print("=" * 50)
        
        # Summary statistics
        self.display_summary_statistics(results, request_type)
        
        # Match score distribution
        print("\n📈 Match Score Distribution:")
        self.create_match_score_distribution(results, f"Match Score Distribution - {request_type.title()}")
        
        # Skills analysis
        print("\n🔧 Skills Analysis:")
        self.create_skills_demand_analysis(results, request_type)
        
        # Location analysis
        print("\n🌍 Geographic Analysis:")
        self.create_location_analysis(results, request_type)
        
        # Experience analysis (if applicable)
        print("\n💼 Experience Analysis:")
        self.create_experience_analysis(results)
        
        print("\n✅ Analytics dashboard generated successfully!")

print("✅ JobMatchingAnalytics class implemented successfully!")
print("📊 Features include: Match score distributions, skills analysis, location mapping, experience analysis, and comprehensive dashboards")

In [ ]:
# Initialize analytics
analytics = JobMatchingAnalytics()

# Enhanced display functions with analytics
def display_job_recommendations_with_analytics(results: Dict):
    """Display job recommendations with analytics"""
    display_job_recommendations(results)
    
    # Add analytics if results are available
    recommendations = results.get("recommendations", {})
    top_matches = recommendations.get("top_matches", [])
    
    if top_matches:
        print("\n" + "="*50)
        print("📊 GENERATING ANALYTICS DASHBOARD")
        print("="*50)
        
        analytics.create_comprehensive_dashboard(
            top_matches, 
            "job_seeker",
            None  # Can pass job seeker requirements here if needed
        )

def display_candidate_recommendations_with_analytics(results: Dict, requirements: Dict = None):
    """Display candidate recommendations with analytics"""
    display_candidate_recommendations(results)
    
    # Add analytics if results are available
    recommendations = results.get("recommendations", {})
    top_matches = recommendations.get("top_matches", [])
    
    if top_matches:
        print("\n" + "="*50)
        print("📊 GENERATING ANALYTICS DASHBOARD")
        print("="*50)
        
        analytics.create_comprehensive_dashboard(
            top_matches, 
            "recruiter",
            requirements
        )

print("✅ Analytics and visualization system implemented successfully!")
print("📊 Features include: Match score distributions, skills analysis, location mapping, experience analysis, and comprehensive dashboards")

## 12. Test the Complete System

Running comprehensive tests to demonstrate the full functionality of the AI job matching agent.

In [ ]:
# Sample test data for demonstrations
SAMPLE_JOB_SEEKER = {
    "name": "Sarah Chen",
    "skills": ["Python", "Machine Learning", "AWS", "Docker", "SQL", "TensorFlow", "Pandas"],
    "experience_years": 5,
    "education": "Masters in Computer Science",
    "location": "San Francisco, CA",
    "desired_salary": "$120,000 - $150,000",
    "job_preferences": ["Senior Data Scientist", "ML Engineer", "AI Researcher"],
    "industry_preferences": ["Technology", "Healthcare", "Finance"],
    "additional_requirements": "Remote work preferred"
}

SAMPLE_JOB_REQUIREMENTS = {
    "job_title": "Senior Full Stack Developer",
    "company_name": "TechCorp Inc.",
    "job_location": "New York, NY (Remote OK)",
    "experience_required": "5+ years",
    "required_skills": ["React", "Node.js", "TypeScript", "AWS", "PostgreSQL"],
    "preferred_skills": ["GraphQL", "Docker", "Kubernetes", "Python"],
    "education_requirements": "Bachelor's degree in Computer Science or related",
    "salary_range": "$130,000 - $160,000",
    "company_culture": "Fast-paced startup environment, collaborative, innovative",
    "special_requirements": "Must be able to work EST hours"
}

async def test_job_seeker_workflow():
    """Test the complete job seeker workflow"""
    
    print("🧪 TESTING JOB SEEKER WORKFLOW")
    print("="*50)
    print(f"Testing with sample profile: {SAMPLE_JOB_SEEKER['name']}")
    print(f"Skills: {', '.join(SAMPLE_JOB_SEEKER['skills'][:5])}...")
    print(f"Experience: {SAMPLE_JOB_SEEKER['experience_years']} years")
    print(f"Location: {SAMPLE_JOB_SEEKER['location']}")
    print("\n🔄 Running job search workflow...")
    
    try:
        # Process the job seeker request
        results = await process_job_seeker_request(SAMPLE_JOB_SEEKER)
        
        print("✅ Job seeker workflow completed successfully!")
        print(f"📊 Status: {results.get('status', 'unknown')}")
        
        # Display results with analytics
        display_job_recommendations_with_analytics(results)
        
        return results
        
    except Exception as e:
        print(f"❌ Error in job seeker workflow: {str(e)}")
        print(f"📝 Error details: {traceback.format_exc()}")
        return None

async def test_recruiter_workflow():
    """Test the complete recruiter workflow"""
    
    print("\n🧪 TESTING RECRUITER WORKFLOW")
    print("="*50)
    print(f"Testing with job requirements: {SAMPLE_JOB_REQUIREMENTS['job_title']}")
    print(f"Company: {SAMPLE_JOB_REQUIREMENTS['company_name']}")
    print(f"Location: {SAMPLE_JOB_REQUIREMENTS['job_location']}")
    print(f"Required Skills: {', '.join(SAMPLE_JOB_REQUIREMENTS['required_skills'])}")
    print("\n🔄 Running candidate search workflow...")
    
    try:
        # Process the recruiter request
        results = await process_recruiter_request(SAMPLE_JOB_REQUIREMENTS)
        
        print("✅ Recruiter workflow completed successfully!")
        print(f"📊 Status: {results.get('status', 'unknown')}")
        
        # Display results with analytics
        display_candidate_recommendations_with_analytics(results, SAMPLE_JOB_REQUIREMENTS)
        
        return results
        
    except Exception as e:
        print(f"❌ Error in recruiter workflow: {str(e)}")
        print(f"📝 Error details: {traceback.format_exc()}")
        return None

async def test_matching_algorithm():
    """Test the matching algorithm with sample data"""
    
    print("\n🧪 TESTING MATCHING ALGORITHM")
    print("="*50)
    
    # Sample candidate data
    sample_candidate = {
        "skills": ["Python", "JavaScript", "React", "SQL", "AWS"],
        "experience": "5 years",
        "location": "San Francisco, CA",
        "education": "Bachelor's in Computer Science"
    }
    
    # Sample job requirements
    sample_requirements = {
        "required_skills": ["Python", "React", "SQL"],
        "preferred_skills": ["AWS", "Docker"],
        "experience_required": "3-5 years",
        "job_location": "San Francisco, CA",
        "education_requirements": "Bachelor's degree"
    }
    
    try:
        # Calculate match
        match_result = matching_engine.calculate_comprehensive_match(
            sample_candidate, 
            sample_requirements
        )
        
        print("✅ Matching algorithm test completed!")
        print(f"📊 Overall Match Score: {match_result.total_score}%")
        print(f"🛠️ Skills Score: {match_result.skills_score}%")
        print(f"📅 Experience Score: {match_result.experience_score}%")
        print(f"📍 Location Score: {match_result.location_score}%")
        print(f"🎓 Education Score: {match_result.education_score}%")
        print(f"💪 Strengths: {', '.join(match_result.strengths) if match_result.strengths else 'None identified'}")
        print(f"📋 Gaps: {', '.join(match_result.gaps) if match_result.gaps else 'None identified'}")
        
        return match_result
        
    except Exception as e:
        print(f"❌ Error in matching algorithm test: {str(e)}")
        return None

def test_search_query_generation():
    """Test search query generation"""
    
    print("\n🧪 TESTING SEARCH QUERY GENERATION")
    print("="*50)
    
    try:
        # Test job seeker queries
        job_seeker = JobSeeker(**SAMPLE_JOB_SEEKER)
        job_queries = search_engine.generate_job_search_queries(job_seeker)
        
        print("✅ Job search queries generated:")
        for i, query in enumerate(job_queries, 1):
            print(f"  {i}. {query}")
        
        # Test recruiter queries
        candidate_queries = search_engine.generate_candidate_search_queries(SAMPLE_JOB_REQUIREMENTS)
        
        print("\n✅ Candidate search queries generated:")
        for i, query in enumerate(candidate_queries, 1):
            print(f"  {i}. {query}")
        
        return {"job_queries": job_queries, "candidate_queries": candidate_queries}
        
    except Exception as e:
        print(f"❌ Error in search query generation test: {str(e)}")
        return None

async def run_comprehensive_system_test():
    """Run comprehensive system test covering all components"""
    
    display(HTML("""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 20px; color: white; margin: 20px 0;">
        <h1 style="margin: 0; text-align: center; color: white;">🚀 AI Job Matching Agent - Comprehensive System Test</h1>
        <p style="text-align: center; margin: 10px 0 0 0; font-size: 1.1em;">Testing all components of the AI-powered job matching system</p>
    </div>
    """))
    
    print("🎯 Starting comprehensive system test...")
    print("⏱️  This test will take several minutes to complete.")
    print("📡 Each test includes real LinkedIn searches via Tavily AI.")
    print("\n" + "="*60)
    
    test_results = {}
    
    # Test 1: Search Query Generation
    print("\n1️⃣ Testing Search Query Generation...")
    query_results = test_search_query_generation()
    test_results["query_generation"] = query_results is not None
    
    # Test 2: Matching Algorithm
    print("\n2️⃣ Testing Matching Algorithm...")
    matching_results = await test_matching_algorithm()
    test_results["matching_algorithm"] = matching_results is not None
    
    # Test 3: Job Seeker Workflow
    print("\n3️⃣ Testing Job Seeker Workflow...")
    job_seeker_results = await test_job_seeker_workflow()
    test_results["job_seeker_workflow"] = job_seeker_results is not None
    
    # Test 4: Recruiter Workflow
    print("\n4️⃣ Testing Recruiter Workflow...")
    recruiter_results = await test_recruiter_workflow()
    test_results["recruiter_workflow"] = recruiter_results is not None
    
    # Final Results
    print("\n" + "="*60)
    print("🏁 COMPREHENSIVE SYSTEM TEST RESULTS")
    print("="*60)
    
    for test_name, success in test_results.items():
        status = "✅ PASSED" if success else "❌ FAILED"
        print(f"{test_name.replace('_', ' ').title()}: {status}")
    
    overall_success = all(test_results.values())
    
    if overall_success:
        display(HTML("""
        <div style="border: 3px solid #2ECC71; padding: 20px; border-radius: 15px; background-color: #D5FFDA; margin: 20px 0;">
            <h2 style="color: #2ECC71; margin: 0; text-align: center;">🎉 ALL TESTS PASSED!</h2>
            <p style="text-align: center; margin: 10px 0 0 0; font-size: 1.1em;">
                The AI Job Matching Agent is fully operational and ready for use!
            </p>
        </div>
        """))
    else:
        display(HTML("""
        <div style="border: 3px solid #E74C3C; padding: 20px; border-radius: 15px; background-color: #FADBD8; margin: 20px 0;">
            <h2 style="color: #E74C3C; margin: 0; text-align: center;">⚠️ SOME TESTS FAILED</h2>
            <p style="text-align: center; margin: 10px 0 0 0;">
                Please check the error messages above and verify your API keys and network connection.
            </p>
        </div>
        """))
    
    return test_results

# Quick test function for individual components
def quick_component_test():
    """Run quick tests for individual components"""
    
    print("⚡ QUICK COMPONENT TEST")
    print("="*30)
    
    # Test data models
    try:
        job_seeker = JobSeeker(**SAMPLE_JOB_SEEKER)
        print("✅ JobSeeker model works")
    except:
        print("❌ JobSeeker model failed")
    
    # Test matching engine
    try:
        matching_engine.calculate_skills_match(["Python", "SQL"], ["Python", "JavaScript"], ["SQL"])
        print("✅ Matching engine works")
    except:
        print("❌ Matching engine failed")
    
    # Test search engine
    try:
        search_engine.categorize_experience(5)
        print("✅ Search engine works")
    except:
        print("❌ Search engine failed")
    
    # Test analytics
    try:
        # Create analytics instance
        analytics = JobMatchingAnalytics()
        result = analytics.extract_years_from_text("5 years experience")
        print(f"✅ Analytics engine works (extracted: {result} years)")
    except Exception as e:
        print(f"❌ Analytics engine failed: {str(e)}")
        # Try to see if class exists
        try:
            print(f"   JobMatchingAnalytics class: {JobMatchingAnalytics}")
        except NameError:
            print("   JobMatchingAnalytics class not found")
    
    print("\n✅ Quick component test completed!")

print("✅ System testing framework implemented successfully!")
print("🧪 Available tests: Comprehensive system test, individual component tests, and workflow validation")
print("🚀 Ready to run full system demonstrations!")

## 🚀 Launch the AI Job Matching Agent

**Ready to use the AI Job Matching Agent? Choose your option below:**

In [ ]:
# 🎯 OPTION 1: Launch Interactive Interface
print("🎯 INTERACTIVE AI JOB MATCHING AGENT")
print("="*40)
print("Choose your role and fill out the form to get started!")
print("The system will search LinkedIn and provide AI-powered recommendations.")
print("\n📱 Click on a role below to begin:")

# Create and display the interactive interface
main_interface = interface.display_mode_selector()
display(main_interface)

# Add the job seeker dashboard
job_seeker_dashboard = create_job_seeker_dashboard()
display(job_seeker_dashboard)

# Add the recruiter dashboard  
recruiter_dashboard = create_recruiter_dashboard()
display(recruiter_dashboard)

### 🧪 Option 2: Run System Tests and Demos

If you want to see the system in action with sample data or test all components:

In [ ]:
# Quick Component Test (Fast)
quick_component_test()

print("\n" + "="*60)
print("🚀 DEMO OPTIONS")
print("="*60)
print("Choose one of the following options to test the system:")
print()
print("1️⃣ Quick Demo - Test core components (30 seconds)")
print("   Run: quick_component_test()")
print()
print("2️⃣ Full System Test - Complete end-to-end test (3-5 minutes)")
print("   Run: await run_comprehensive_system_test()")
print()
print("3️⃣ Job Seeker Demo - Test job search with sample data (1-2 minutes)")
print("   Run: await test_job_seeker_workflow()")
print()
print("4️⃣ Recruiter Demo - Test candidate search with sample data (1-2 minutes)")
print("   Run: await test_recruiter_workflow()")
print()
print("💡 Tip: Copy and paste the 'Run:' commands into new cells to execute them!")
print("📝 Note: Tests with 'await' need to be run in async context or new cells.")

# Convenient test runners
print("\n🎮 INTERACTIVE TEST LAUNCHER")
print("="*30)

def create_test_buttons():
    """Create interactive test buttons"""
    
    # Test buttons
    quick_test_btn = widgets.Button(
        description='⚡ Quick Test',
        button_style='info',
        layout=widgets.Layout(width='150px', height='40px', margin='5px')
    )
    
    full_test_btn = widgets.Button(
        description='🧪 Full Test',
        button_style='primary', 
        layout=widgets.Layout(width='150px', height='40px', margin='5px')
    )
    
    job_demo_btn = widgets.Button(
        description='👤 Job Seeker Demo',
        button_style='success',
        layout=widgets.Layout(width='150px', height='40px', margin='5px')
    )
    
    recruiter_demo_btn = widgets.Button(
        description='👥 Recruiter Demo',
        button_style='warning',
        layout=widgets.Layout(width='150px', height='40px', margin='5px')
    )
    
    output = widgets.Output()
    
    def on_quick_test(b):
        with output:
            clear_output()
            quick_component_test()
    
    def on_full_test(b):
        with output:
            clear_output()
            print("🚀 Starting full system test...")
            print("📝 Run this in a new cell: await run_comprehensive_system_test()")
    
    def on_job_demo(b):
        with output:
            clear_output()
            print("👤 Starting job seeker demo...")
            print("📝 Run this in a new cell: await test_job_seeker_workflow()")
    
    def on_recruiter_demo(b):
        with output:
            clear_output()
            print("👥 Starting recruiter demo...")
            print("📝 Run this in a new cell: await test_recruiter_workflow()")
    
    quick_test_btn.on_click(on_quick_test)
    full_test_btn.on_click(on_full_test)
    job_demo_btn.on_click(on_job_demo)
    recruiter_demo_btn.on_click(on_recruiter_demo)
    
    return widgets.VBox([
        widgets.HTML("<p>Click a button to run tests:</p>"),
        widgets.HBox([quick_test_btn, full_test_btn]),
        widgets.HBox([job_demo_btn, recruiter_demo_btn]),
        output
    ])

# Display test buttons
test_interface = create_test_buttons()
display(test_interface)

In [ ]:
await test_job_seeker_workflow()

In [ ]:
await test_recruiter_workflow()

## 🎉 Congratulations!

You have successfully implemented a complete **AI Job Matching Agent** that includes:

### ✅ **Core Features Implemented:**

- **🤖 AI-Powered Matching**: Uses OpenAI GPT-4o-mini for intelligent analysis
- **🔍 LinkedIn Search**: Leverages Tavily AI for comprehensive LinkedIn searches
- **📊 Advanced Analytics**: Interactive visualizations with Plotly
- **🎯 Dual Functionality**: Serves both job seekers and recruiters
- **⚡ LangGraph Workflow**: Orchestrates complex multi-step processes
- **📈 Match Scoring**: Sophisticated algorithms for compatibility assessment
- **🎨 Interactive UI**: User-friendly forms and result displays
- **📊 Rich Visualizations**: Charts, graphs, and analytics dashboards

### 🚀 **Next Steps:**

1. **Use the Interactive Interface**: Fill out the forms above to search for jobs or candidates
2. **Run the Demos**: Execute the test functions to see sample results
3. **Customize the System**: Modify prompts, weights, or search strategies
4. **Scale Up**: Add more data sources, enhance matching algorithms, or integrate with ATS systems

### 💡 **Tips for Best Results:**

- Use specific, relevant skills in your searches
- Include location preferences for better matching
- Experiment with different search terms and requirements
- Review the analytics to understand market trends

### 📚 **What You've Learned:**

- How to integrate multiple AI services (OpenAI, Tavily AI, LangChain)
- Building complex workflows with LangGraph
- Creating interactive Jupyter interfaces
- Implementing matching algorithms and analytics
- Real-world AI application development

---

**🌟 Your AI Job Matching Agent is ready to help connect the right people with the right opportunities!**